# Cleaning Stock Market Data Scraped Off the Web

In a previous project, I used Beautiful Soup and the Requests library to scape stock market data from the finviz website, then store the data in csv file.

Since I'd like to analyze this data, I'll use the Pandas and Numpy libraries to clean the dataset so that all columns are formatted properly. By the end of this project, all numeric variables will have the floating point or integer data types. I'll also fix and create some new indicator variables.

## Load the Data

Before I load the data, I'll import the Pandas and Numpy libraries.

In [1]:
import pandas as pd
import numpy as np

Next, I'll read the dataset and print the data type for each column.

In [2]:
sp = pd.read_csv("S&P500.csv",index_col=0) # TICKER SYMBOLS ARE INDEX
sp.dtypes

Index             object
P/E               object
EPS (ttm)        float64
Insider Own       object
Shs Outstand      object
Perf Week         object
Market Cap        object
Forward P/E       object
EPS next Y        object
Insider Trans     object
Shs Float         object
Perf Month        object
Income            object
PEG               object
EPS next Q        object
Inst Own          object
Short Float       object
Perf Quarter      object
Sales             object
P/S               object
EPS this Y        object
Inst Trans        object
Short Ratio      float64
Perf Half Y       object
Book/sh          float64
P/B               object
EPS next Y.1      object
ROA               object
Target Price      object
Perf Year         object
                  ...   
Dividend %        object
Quick Ratio       object
Sales past 5Y     object
Gross Margin      object
52W Low           object
ATR              float64
Employees          int64
Current Ratio     object
Sales Q/Q         object


Many of the columns, which should be numeric, have dtype = object.

Let's look at the first 5 rows to get a better idea of what I need to do.

In [3]:
sp.head()

,Index,P/E,EPS (ttm),Insider Own,Shs Outstand,Perf Week,Market Cap,Forward P/E,EPS next Y,Insider Trans,...,Earnings,Payout,Avg Volume,Price,Recom,SMA20,SMA50,SMA200,Volume,Change
A,S&P; 500,24.42,2.68,0.30%,321.92M,-1.53%,21.07B,19.41,3.37,-5.17%,...,Nov 19 AMC,58.10%,2.85M,65.46,1.80,-3.54%,-2.13%,-1.22%,"3,119,738",3.46%
AA,S&P; 500,977.24,0.03,0.10%,195.69M,4.34%,5.55B,8.01,3.54,0.00%,...,Jan 16 AMC,-,3.69M,28.34,2.00,2.06%,-10.95%,-33.25%,"5,652,866",8.00%
AAL,S&P; 500,6.94,4.62,0.20%,477.01M,0.00%,15.28B,5.63,5.69,4.38%,...,Jan 24 BMO,13.80%,10.05M,32.04,2.00,-2.25%,-7.81%,-19.46%,"9,344,567",6.59%
AAP,S&P; 500,34.34,4.63,0.10%,74.11M,2.66%,11.77B,19.50,8.14,0.27%,...,Feb 12 BMO,3.20%,1.19M,158.81,2.30,-1.36%,-5.01%,8.71%,"1,423,037",-2.48%
AAPL,DJIA S&P500;,12.21,12.15,0.07%,4.87B,-5.05%,722.24B,10.13,14.64,-14.91%,...,Jan 31 AMC,22.70%,43.57M,148.26,2.10,-8.38%,-19.21%,-23.19%,"57,275,863",4.27%


## Clean the Data

In no particular order, here are the problems I'm seeing:

* Percentages have a percent symbol.
* Very large numbers have a denomination symbol, e.g. B for billion.
* The Earnings column has text at the end of each date.
* The Volatility column contains 2 columns of information.
* The Volume column has commas in it.
* The Optionable and Shortable columns take on the values "Yes" or "No", rather than 1 or 0.
* Missing values are indicated with a "-" symbol.

In the next sections, I'll fix all of these issues.

To start, I'm going to make a copy of the original dataset.

In [4]:
clean_sp = sp.copy()

### Remove the Percent Signs

To remove the percent sign from columns with percentages, I'm going to use the apply method. With the apply method, I'll apply an anonymous function to each column with percentage symbols. You can read more about the apply method [here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html).

In [5]:
# remove percent symbol
percent_columns = ['Insider Own','Perf Week',
                   'Insider Trans','Perf Month',
                   'Inst Own','Short Float','Perf Quarter',
                   'EPS this Y','Inst Trans','Perf Half Y',
                   'EPS next Y.1','ROA','Perf Year',
                   'EPS next 5Y','ROE','Perf YTD',
                   'EPS past 5Y','ROI','52W High','Dividend %',
                   'Sales past 5Y','Gross Margin',
                   '52W Low','Sales Q/Q','Oper. Margin',
                   'Volatility','EPS Q/Q','Profit Margin',
                   'Payout','SMA20','SMA50','SMA200',
                   'Change']

clean_sp[percent_columns] = sp[percent_columns].apply(lambda x: x.str.replace('%',''))

### Remove the Denomination Symbols

Removing the denomination symbols won't be as easy. My solution is to create a function to handle the conversion, then apply it element-wise, using the applymap method. Thus unlike the apply method, the applymap method applies a function to each cell of the data frame. You can read more about the applymap method [here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.applymap.html).

In [6]:
denomination_columns = ['Shs Outstand','Market Cap','Shs Float','Income',
                           'Sales','Avg Volume']

def actual_number(string):
    if string == '-':
        return np.NaN
    denomination = string[-1]
    string = string[0:len(string)-1]
    string = float(string)
    if denomination == 'B':
        string = string * 1000000000
    elif denomination == 'M':
        string = string * 1000000
    elif denomination == 'K':
        string = string * 1000
    return string
    

clean_sp[denomination_columns] = sp[denomination_columns].applymap(actual_number)

### Remove Commas

To remove the commas from the Volume column, I'll use the str.replace method. Feel free to read more about it [here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.replace.html).

In [7]:
clean_sp['Volume'] = sp['Volume'].str.replace(',','')

### Removing a Redundant Column

Since there's already a 52 Week High and 52 Week Low, I'm thinking that a 52 Week Range, which is the difference between the 2 columns, is a little redundant. If I really need a 52 Week Range, I can always calculate, but for now, I'll drop the column using the drop method. You can read more about the drop method [here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html).

In [8]:
clean_sp.drop(['52W Range'],axis=1,inplace=True)

### Fixing the Volatility Column

After reading the finviz screener [help menu](https://finviz.com/help/screener.ashx), I found that the Volatility column really contains two measures, the average daily high range, and the average daily low range. I'll create 2 new columns for these measures, by splitting the Volatility column with the str.split method, then drop the original column from the dataset. You can read more about the str.split method [here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.split.html).

In [9]:
volatility = clean_sp['Volatility'].str.split(expand=True)
clean_sp['avg_daily_high_range'] = volatility.iloc[:,0]
clean_sp['avg_daily_low_range'] = volatility.iloc[:,1]
clean_sp.drop(['Volatility'],axis=1,inplace=True)

In case I need to return to my list of percent columns, I'll remove Volatility, and add the 2 other columns.

In [10]:
percent_columns.remove('Volatility')
percent_columns.append('avg_daily_high_range')
percent_columns.append('avg_daily_low_range')

### Fixing the Earnings Column

The earnings column has the symbol BMO or AMC appended at the end. BMO means the earnings are released before the market opens. AMC means the earnings are released after the market closes. Using the str.contains and astype methods, I'll create indicator variables for both scenarios. You can read more about the str.contains method [here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.contains.html) and the astype method [here](https://pandas.pydata.org/pandas-docs/version/0.18/generated/pandas.Series.astype.html).

In [11]:
#convert earnings before market open to an indicator
clean_sp['Earnings_BMO'] = sp['Earnings'].str.contains('BMO').astype('int')
#convert earnings after market close to an indicator
clean_sp['Earnings_AMC'] = sp['Earnings'].str.contains('AMC').astype('int')

With these indicators created, I'll use the str.replace, to_datetime and apply methods to convert the Earnings columns to a date with the year 2019. You can read more about str.replace method [here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.replace.html) and the to_dateime method [here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html).

In [12]:
clean_sp['Earnings'] = sp['Earnings'].str.replace('BMO','').str.replace('AMC','')
clean_sp['Earnings'] = clean_sp['Earnings'].str.replace(' ','')
clean_sp['Earnings'] = pd.to_datetime(clean_sp['Earnings'],format = '%b%d')
clean_sp['Earnings'] = clean_sp['Earnings'].apply(lambda x: x.replace(year=2019))

### Fixing the Optionable and Shortable Columns

I'd prefer to have a column taking on the values 0 or 1, rather than 'No' or 'Yes'.

In [13]:
# change optional to an indicator
sp['Optionable'].value_counts()
clean_sp['Optionable'] = (sp['Optionable'] == 'Yes').astype('int')

# change shortable to an indicator
sp['Shortable'].value_counts()
clean_sp['Shortable'] = (sp["Shortable"] == 'Yes').astype('int')

### Creating a DJIA Column

Since a few S&P 500 stocks are members of the Dow Jones Industrial Average (DJIA), I'll create an indicator for DJIA membership.

In [14]:
# make an indicator for Dow membership
sp['Index'].value_counts()
clean_sp['DJIA'] = sp['Index'].str.contains('DJIA').astype('int')
clean_sp.drop(["Index"],axis=1,inplace=True)

### Replacing '-' Signs with NaNs

The string columns with missing data have a "-" sign. I'll convert these to Na's by creating a custom function and applying it to each data-element with the applymap method.

In [15]:
string_columns = clean_sp.select_dtypes(include=['object']).columns.tolist()

def replace_minus(string):
    if string == '-':
        return np.NaN
    else:
        return float(string)
    
clean_sp.loc[:,string_columns] = clean_sp.loc[:,string_columns].applymap(replace_minus)

Now, every cell is numeric. There is just 1 cleaning step for me to do.

### Converting Percentages to Numbers

Every column is numeric, but the percentages aren't scaled correctly. I'll divide all percentages by 100, so it's clear that they are fractions now.

In [16]:
clean_sp[percent_columns] = clean_sp[percent_columns] / 100

## Conclusion

The dataset has been cleaned, more or less. Take a look.

In [17]:
clean_sp.head()

,P/E,EPS (ttm),Insider Own,Shs Outstand,Perf Week,Market Cap,Forward P/E,EPS next Y,Insider Trans,Shs Float,...,SMA20,SMA50,SMA200,Volume,Change,avg_daily_high_range,avg_daily_low_range,Earnings_BMO,Earnings_AMC,DJIA
A,24.42,2.68,0.0030,3.219200e+08,-0.0153,2.107000e+10,19.41,3.37,-0.0517,3.178900e+08,...,-0.0354,-0.0213,-0.0122,3119738.0,0.0346,0.0296,0.0314,0,1,0
AA,977.24,0.03,0.0010,1.956900e+08,0.0434,5.550000e+09,8.01,3.54,0.0000,1.859900e+08,...,0.0206,-0.1095,-0.3325,5652866.0,0.0800,0.0522,0.0511,0,1,0
AAL,6.94,4.62,0.0020,4.770100e+08,0.0000,1.528000e+10,5.63,5.69,0.0438,4.575900e+08,...,-0.0225,-0.0781,-0.1946,9344567.0,0.0659,0.0554,0.0530,1,0,0
AAP,34.34,4.63,0.0010,7.411000e+07,0.0266,1.177000e+10,19.50,8.14,0.0027,7.268000e+07,...,-0.0136,-0.0501,0.0871,1423037.0,-0.0248,0.0318,0.0320,1,0,0
AAPL,12.21,12.15,0.0007,4.870000e+09,-0.0505,7.222400e+11,10.13,14.64,-0.1491,4.740000e+09,...,-0.0838,-0.1921,-0.2319,57275863.0,0.0427,0.0267,0.0341,0,1,1


With this project done, I'll store the cleaned up dataset as a .csv for later use.

In [18]:
clean_sp.to_csv("clean_sp500.csv",index=True)

In the next project, I plan on doing some exploratory analysis.

I hope you like the project. Thanks for reading.